In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, RNN

In [2]:
text = "Hello World"
chars = sorted(set(text))
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for c, i in char_to_int.items()}

In [3]:
print(char_to_int)

{' ': 0, 'H': 1, 'W': 2, 'd': 3, 'e': 4, 'l': 5, 'o': 6, 'r': 7}


In [4]:
seq_length = 4
X = []
y = []

for i in range(len(text) - seq_length):
    seq_in = text[i:i+seq_length]
    seq_out = text[i + seq_length]
    X.append([char_to_int[char] for char in seq_in])
    y.append(char_to_int[seq_out])

X = np.reshape(X, (len(X), seq_length, 1)) / float(len(chars))
y = np.eye(len(chars))[y]  # One-hot encode output

In [5]:
print("X:", X.shape)

X: (7, 4, 1)


In [6]:
class MyRNNCell(tf.keras.layers.Layer):
    def __init__(self, rnn_units, input_dim, output_dim):
        super(MyRNNCell, self).__init__()
        self.rnn_units = rnn_units
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.state_size = rnn_units
        self.output_size = output_dim

        self.W_xh = self.add_weight(shape=(rnn_units, input_dim), initializer="random_normal")
        self.W_hh = self.add_weight(shape=(rnn_units, rnn_units), initializer="random_normal")
        self.W_hy = self.add_weight(shape=(output_dim, rnn_units), initializer="random_normal")

    def call(self, inputs, states):
        prev_h = states[0]  # (batch_size, rnn_units)

        # matrix multiplications with batch support
        h = tf.math.tanh(
            tf.matmul(prev_h, self.W_hh, transpose_b=True) +
            tf.matmul(inputs, self.W_xh, transpose_b=True)
        )  # shape: (batch_size, rnn_units)

        output = tf.matmul(h, self.W_hy, transpose_b=True)  # shape: (batch_size, output_dim)

        return output, [h]


In [7]:
cell = MyRNNCell(rnn_units=32, input_dim=X.shape[2], output_dim=8)

In [8]:
model = Sequential()
model = tf.keras.Sequential([
    tf.keras.layers.RNN(cell, input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/Users/anveshradharapu/Documents/Project_Space/ML Engineer/NadaDotAI/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rnn (RNN)                       │ (None, 8)              │         1,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │            72 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,384 (5.41 KB)

 Trainable params: 1,384 (5.41 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.fit(X, y, epochs=300, verbose=0)

In [12]:
seed = "Hol"
pattern = [char_to_int[c] for c in seed]
for _ in range(10):
    x = np.reshape(pattern, (1, len(pattern), 1)) / float(len(chars))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    print(result, end='')
    pattern.append(index)
    pattern = pattern[1:]

looooooooo